使用許諾条件

①	設計一次エネルギー消費量算定法に基づく計算方法をPythonによりコーディングし、Jupyter notebook形式で保存したファイル（以下、本ファイルと呼ぶ）の著作権は、一般社団法人日本サステナブル建築協会に帰属します。
Copyright©2019. 一般社団法人日本サステナブル建築協会
All rights reserved. 

②	本ファイルは、住戸の設計一次エネルギー消費量算定方法に係る「平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）2.1算定方法（第三章第二節 外皮性能）」に示す計算方法に基づき、国土交通省国土技術政策総合研究所・国立研究開発法人建築研究所、株式会社ユーワークス、株式会社住環境計画研究所が作成したものです。

③	本ファイルは、計算方法に示す計算過程の確認及び調査研究等に活用することを目的として作成されたものであり、一般社団法人日本サステナブル建築協会は著作権について放棄はしませんが、使用する各人の自己責任において仕様を変更することを妨げるものではありません。ただし、本ファイルによる計算結果を住宅の省エネルギー基準で規定する設計一次エネルギー消費量の結果として公的な申請に用いることはできません。

④	本ファイルを変更するかしないかを問わず、以下の条件を満たす場合に限り、頒布および使用が許可されます。
1.	頒布物の付属ドキュメント等の資料に、上記の著作権表示、本条件一覧、および下記免責条項を含めること。
2.	書面による特別の許可なしに、本ファイルから派生した製品の宣伝または販売の促進に、一般社団法人日本サステナブル建築協会、国土技術政策総合研究所・国立研究開発法人建築研究所および株式会社ユーワークス、株式会社住環境計画研究所の名前またはコントリビューターの名前を使用してはならない。

⑤	本ファイルは、著作権者およびコントリビューターによって「現状のまま」提供されており、明示黙示を問わず、商業的な使用可能性、および特定の目的に対する適合性に関する暗黙の保証も含め、またそれに限定されない、いかなる保証もありません。著作権もコントリビューターも、事由のいかんを問わず、損害発生の原因いかんを問わず、かつ責任の根拠が契約であるか厳格責任であるか（過失その他の）不法行為であるかを問わず、仮にそのような損害が発生する可能性を知らされていたとしても、本ファイルの使用によって発生した（代用品または代用サービスの調達、使用の喪失、データの喪失、利益の喪失、業務の中断も含め、またそれに限定されない）直接損害、間接損害、偶発的な損害、特別損害、懲罰的損害、または結果損害について、一切の責任を負わないものとします。



In [ ]:
import nbimporter
import section3_2_8 as detail
import section3_2_9 as simple


## 第二節　外皮性能 / Section 2 Envelope Performance

1．適用範囲～4．記号及び単位は省略

1. Scope of application ~ 4. Symbols and units are omitted

In [ ]:
def calc_insulation_performance(method, A_env=None, A_A=None, U_A=None, eta_A_H=None, eta_A_C=None,
                                house_insulation_type=None, insulation_structure=None,
                                U_spec=None,f_H=None,f_C=None,eta_d_H=None,eta_d_C=None,region=None):
    """ 外皮の断熱性能の計算
     入力方法によって、U_A, eta_A_H, eta_A_C, r_env の計算方法が異なる
     1.当該住宅の外皮面積の合計を用いて評価する => すべて別途計算された結果を用いる
     2.簡易的に求めた外皮面積の合計を用いて評価する => U_A, eta_A_H, eta_A_Cは別途計算された値を用いるが、r_envは簡易計算
     3.当該住戸の外皮の部位の面積等を用いて外皮性能を評価する方法 => すべて簡易計算

    :param method: 入力方法
    :type method: str
    :param A_env: 外皮の部位の面積の合計 (m2)
    :type A_env: float
    :param A_A: 床面積の合計[m^2]
    :type A_A: float
    :param U_A: 外皮平均熱貫流率
    :type U_A: float
    :param eta_A_H: 暖房期の平均日射熱取得率
    :type eta_A_H: float
    :param eta_A_C: 冷房期の平均日射熱取得率
    :type eta_A_C: float
    :param house_insulation_type: '床断熱住戸'または'基礎断熱住戸'
    :type house_insulation_type: str
    :param insulation_structure: 断熱構造による住戸の種類
    :type insulation_structure: str
    :param U_spec: 外皮の部位の熱貫流率の辞書
    :type U_spec: dict
    :param f_H: 暖房期の取得日射熱補正係数 (-)
    :type f_H: float
    :param f_C: 冷房期の取得日射熱補正係数 (-)
    :type f_C: float
    :param eta_d_H: 暖房期の日射熱取得率
    :type eta_d_H:
    :param eta_d_C:
    :type eta_d_C:
    :param region: 省エネルギー地域区分
    :type region: int
    :return: 外皮の断熱性能
    :rtype: tuple
    """
    if method == '当該住宅の外皮面積の合計を用いて評価する':
        # 床面積の合計に対する外皮の部位の面積の合計の比
        r_env = calc_r_env(
            method='当該住戸の外皮の部位の面積等を用いて外皮性能を評価する方法',
            A_env=A_env,
            A_A=A_A
        )
    elif method == '簡易的に求めた外皮面積の合計を用いて評価する':
        # 床面積の合計に対する外皮の部位の面積の合計の比
        r_env = calc_r_env(
            method='当該住戸の外皮の部位の面積等を用いずに外皮性能を評価する方法',
            house_insulation_type=house_insulation_type
        )
    elif method == '当該住戸の外皮の部位の面積等を用いて外皮性能を評価する方法':
        # 断熱構造による住戸の種類に応じてU_A値を計算する
        U_A, U = simple.calc_U_A(insulation_structure, **U_spec)

        # 床面積の合計に対する外皮の部位の面積の合計の比
        r_env = calc_r_env(
            method='当該住戸の外皮の部位の面積等を用いずに外皮性能を評価する方法',
            A_env=None,
            A_A=None,
            house_insulation_type=U['house_insulation_type']
        )

        # 暖房期平均日射熱取得率(ηAH)
        eta_A_H = calc_eta_A_H(
            method='当該住戸の外皮の部位の面積等を用いずに外皮性能を評価する方法',
            region=region,
            house_insulation_type=U['house_insulation_type'],
            U_roof=U['U_roof'],
            U_wall=U['U_wall'],
            U_door=U['U_door'],
            U_base_etrc=U['U_base_etrc'],
            U_base_bath=U['U_base_bath'],
            U_base_other=U['U_base_other'],
            etr_d=eta_d_H,
            f_H=f_H
        )

        # 冷房期平均日射熱取得率(ηAC)
        eta_A_C = calc_eta_A_C(
            method='当該住戸の外皮の部位の面積等を用いずに外皮性能を評価する方法',
            region=region,
            house_insulation_type=U['house_insulation_type'],
            U_roof=U['U_roof'],
            U_wall=U['U_wall'],
            U_door=U['U_door'],
            U_base_etrc=U['U_base_etrc'],
            U_base_bath=U['U_base_bath'],
            U_base_other=U['U_base_other'],
            etr_d=eta_d_C,
            f_C=f_C
        )

        house_insulation_type = U['house_insulation_type']
    else:
        raise ValueError(method)

    # 熱損失係数（換気による熱損失を含まない）
    Q_dash = get_Q_dash(U_A, r_env)

    # 日射取得係数
    eta_H = get_eta_H(eta_A_H, r_env)
    eta_C = get_eta_C(eta_A_C, r_env)

    return U_A, r_env, eta_A_H, eta_A_C, Q_dash, eta_H, eta_C, house_insulation_type


### 5. 熱損失係数（換気による熱損失を含まない） / Heat loss coefficient (does not include heat loss due to ventilation)

$$\large
Q' = U_{A} \times r_{env}
$$

$Q'$：熱損失係数（換気による熱損失を含まない）(W/m<sup>2</sup>K) / Heat loss coefficient (does not include heat loss due to ventilation)  
$U_{A}$：外皮平均熱貫流率(W/m<sup>2</sup>K) / Average heat penetration coefficient of skin  
$r_{env}$：床面積の合計に対する外皮の部位の面積の合計の比 / Ratio of the sum of the area of the area of the hull to the sum of the floor area 

In [ ]:
def get_Q_dash(U_A, r_env):
    """ 熱損失係数（換気による熱損失を含まない） (1)

    :param U_A: 外皮平均熱貫流率
    :type U_A: float
    :param r_env:床面積の合計に対しる外皮の部位の面積の合計の比（-）
    :type r_env: float
    :return: 熱損失係数（換気による熱損失を含まない）
    :rtype: float
    """
    return U_A * r_env  # (1)


### 6. 日射取得係数 / Solar radiation acquisition coefficient

$$\large
\mu_{H} = \frac{\eta_{A,H}}{100} \times r_{env}
$$

$$\large
\mu_{C} = \frac{\eta_{A,C}}{100} \times r_{env}
$$

$\mu_{H}$：暖房期の日射取得係数(-) / Solar radiation acquisition factor of heating season  
$\mu_{C}$：冷房期の日射取得係数(-) / Solar radiation acquisition coefficient during the cooling period  
$\eta_{A,H}$：暖房期の平均日射熱取得率(％) / Average solar heat gain rate during the heating season  
$\eta_{A,C}$：冷房期の平均日射熱取得率(％) / Average solar heat gain rate during the cooling period  
$r_{env}$：床面積の合計に対する外皮の部位の面積の合計の比 / Ratio of the total area of the area of the hull to the total floor area

In [ ]:
def get_eta_H(eta_A_H, r_env):
    """ 暖房期の日射取得係数 (2)

    :param eta_A_H: 暖房期の平均日射熱取得率
    :type eta_A_H: float
    :param r_env:床面積の合計に対しる外皮の部位の面積の合計の比（-）
    :type r_env: float
    :return: 暖房期の日射取得係数
    :rtype: float
    """
    if eta_A_H is None:
        return None
    return eta_A_H / 100.0 * r_env


In [ ]:
def get_eta_C(eta_A_C, r_env):
    """ 冷房期の日射取得係数 (3)

    :param eta_A_C: 冷房期の平均日射熱取得率
    :type eta_A_C: float
    :param r_env:床面積の合計に対しる外皮の部位の面積の合計の比（-）
    :type r_env: float
    :return: 冷房期の日射取得係数
    :rtype: float
    """
    return eta_A_C / 100.0 * r_env


### 7. 外皮平均熱貫流率並びに暖房期及び冷房期の平均日射熱取得率 / Skin average heat transmission rate and average solar heat gain rate during heating and cooling periods

省略

ommision

In [ ]:
def calc_U_A(method, **args):
    """ 外皮平均熱貫流率

    :param method: 入力方法
    :type method: str
    :param args:
    :type args:
    :return: 外皮平均熱貫流率
    :rtype: float
    """
    if method == '当該住戸の外皮の部位の面積等を用いて外皮性能を評価する方法':
        return detail.get_U_A(**args)
    elif method == '当該住戸の外皮の部位の面積等を用いずに外皮性能を評価する方法':
        return simple.get_U_A(**args)
    else:
        raise ValueError(method)


In [ ]:
def calc_eta_A_H(method, region, house_insulation_type, U_roof, U_wall, U_door, U_base_etrc, U_base_bath,
            U_base_other, etr_d, f_H):
    """ 暖房期の平均日射熱取得率

    :param method: 入力方法
    :type method: str
    :param region: 省エネルギー地域区分
    :type region: int
    :param house_insulation_type: '床断熱住戸'または'基礎断熱住戸'
    :type house_insulation_type: str
    :param U_roof: 屋根又は天井の熱貫流率
    :type U_roof: float
    :param U_wall: 壁の熱貫流率
    :type U_wall: float
    :param U_door: ドアの熱貫流率
    :type U_door: float
    :param U_base_etrc: 玄関等の基礎の熱貫流率
    :type U_base_etrc: float
    :param U_base_bath: 浴室の基礎の熱貫流率
    :type U_base_bath: float
    :param U_base_other: その他の基礎の熱貫流率
    :type U_base_other: float
    :param etr_d: 暖房期の垂直面日射熱取得率 (-)
    :type etr_d: float
    :param f_H: 暖房期の取得日射熱補正係数 (-)
    :type f_H: float
    :return: 暖房期の平均日射熱取得率
    :rtype: float
    """

    if method == '当該住戸の外皮の部位の面積等を用いて外皮性能を評価する方法':
        return detail.get_eta_A_H()
    elif method == '当該住戸の外皮の部位の面積等を用いずに外皮性能を評価する方法':

        # 単位日射強度当たりの暖房期の日射熱取得量[W/(W/m2)]
        m_H = simple.get_m_H(
            region=region,
            house_insulation_type=house_insulation_type,
            U_roof=U_roof,
            U_wall=U_wall,
            U_door=U_door,
            U_base_etrc=U_base_etrc,
            U_base_bath=U_base_bath,
            U_base_other=U_base_other,
            etr_d=etr_d,
            f_H=f_H
        )

        A_dash_env = simple.get_A_dash_env(house_insulation_type)

        return simple.get_eta_A_H(m_H=m_H, A_dash_env=A_dash_env)
    else:
        raise ValueError(method)

In [ ]:
def calc_eta_A_C(method, region, house_insulation_type, U_roof, U_wall, U_door, U_base_etrc, U_base_bath, U_base_other,
            etr_d, f_C):
    """ 冷房期の平均日射熱取得率

    :param method: 入力方法
    :type method: str
    :param region: 省エネルギー地域区分
    :type region: int
    :param house_insulation_type: '床断熱住戸'または'基礎断熱住戸'
    :type house_insulation_type: str
    :param U_roof: 屋根又は天井の熱貫流率
    :type U_roof: float
    :param U_wall: 壁の熱貫流率
    :type U_wall: float
    :param U_door: ドアの熱貫流率
    :type U_door: float
    :param U_base_etrc: 玄関等の基礎の熱貫流率
    :type U_base_etrc: float
    :param U_base_bath: 浴室の基礎の熱貫流率
    :type U_base_bath: float
    :param U_base_other: その他の基礎の熱貫流率
    :type U_base_other: float
    :param etr_d: 暖房期の垂直面日射熱取得率 (-)
    :type etr_d: float
    :param f_C: 冷房期の取得日射熱補正係数 (-)
    :type f_C: float
    :return: 冷房期の平均日射熱取得率
    :rtype: float
    """

    if method == '当該住戸の外皮の部位の面積等を用いて外皮性能を評価する方法':
        return detail.get_eta_A_C()
    elif method == '当該住戸の外皮の部位の面積等を用いずに外皮性能を評価する方法':

        # 単位日射強度当たりの冷房期の日射熱取得量[W/(W/m2)]
        m_C = simple.get_m_C(
            region=region,
            house_insulation_type=house_insulation_type,
            U_roof=U_roof,
            U_wall=U_wall,
            U_door=U_door,
            U_base_etrc=U_base_etrc,
            U_base_bath=U_base_bath,
            U_base_other=U_base_other,
            etr_d=etr_d,
            f_C=f_C)

        A_dash_env = simple.get_A_dash_env(house_insulation_type)

        return simple.get_eta_A_C(m_C=m_C, A_dash_env=A_dash_env)
    else:
        raise ValueError(method)


In [ ]:
def calc_r_env(method, A_env=None, A_A=None, house_insulation_type=None):
    """ 床面積の合計に対する外皮の部位の面積の合計の比

    :param method: 入力方法
    :type method: str
    :param A_env: 外皮の部位の面積の合計 (m2)
    :type A_env: float
    :param A_A: 床面積の合計[m^2]
    :type A_A: float
    :param house_insulation_type: '床断熱住戸'または'基礎断熱住戸'
    :type house_insulation_type: str
    :return: 床面積の合計に対する外皮の部位の面積の合計の比
    :rtype: float
    """
    if method == '当該住戸の外皮の部位の面積等を用いて外皮性能を評価する方法':
        return detail.get_r_env(
            A_env=A_env,
            A_A=A_A
        )
    elif method == '当該住戸の外皮の部位の面積等を用いずに外皮性能を評価する方法':

        # 外部部位の面積の合計 (m2)
        A_dash_env = simple.get_A_dash_env(house_insulation_type)

        # 床面積の合計
        A_dash_A = simple.get_A_dash_A()

        return simple.get_r_env(
            A_dash_env=A_dash_env,
            A_dash_A=A_dash_A
        )
    else:
        raise ValueError(method)


## 付録 A 外皮の部位の面積及び熱橋等の長さ等 / Appendix A Area of hull part and length of heat bridge etc.

省略

ommision

## 付録 B 温度差係数 / Appendix B Temperature difference coefficient

<div style="text-align: center;font-weight: bold;">表　温度差係数<br>Table: Temperature difference coefficient</div>

<table>
<thead>
<tr>
<th colspan="4" style="text-align:center;">隣接空間の種類<br>Adjacent space type</th>
</tr>
<tr>
<th rowspan="2" style="text-align:center;">外気<br>外気に通じる空間<br>Outside air<br>Space that leads to the open air</th>
<th rowspan="2" style="text-align:center;">外気に通じていない空間<br>外気に通じる床裏<br>Space that does not communicate outside<br>The floor is open to the atmosphere</th>
<th colspan="2" style="text-align:center;">住戸及び住戸と同様の熱的環境の空間<br>外気に通じていない床裏<br>Space of thermal environment similar to dwelling unit and dwelling unit<br>The floor does not communicate with the outside air</th>
</tr>
<tr>
<th>1～3 地域<br>1 to 3 area</th>
<th>4～8 地域<br>4 to 8 area</th>
</tr>
</thead>

<tbody>
<tr>
<td>1.0</td><td>0.7</td><td>0.05</td><td>0.15</td>
</tr>
</tbody>
</table>


In [ ]:
# 温度差係数
def get_H(adjacent_type, region):
    """
    :param adjacent_type: 隣接空間の種類
    :param region: 地域区分
    :return: 温度差係数 (-)
    """
    # 表1 温度差係数
    table_1 = (1.0, 0.7, 0.05, 0.15)
    if adjacent_type == '外気・外気に通じる空間':
        return table_1[0]
    elif adjacent_type == '外気に通じていない空間・外気に通じる床裏':
        return table_1[1]
    elif adjacent_type == '住戸及び住戸と同様の熱的環境の空間・外気に通じていない床裏':
        if region in [1, 2, 3]:
            return table_1[2]
        elif region in [4, 5, 6, 7, 8]:
            return table_1[3]
        else:
            raise ValueError(region)
    else:
        raise ValueError(adjacent_type)

## 付録  C 方位係数 / Appendix C Orientation coefficient

界壁及び界床の方位係数は0とする。

The orientation coefficient of the boundary wall and the boundary floor is 0.

In [ ]:
# 界壁及び界床の方位係数 (-)
def get_nu_boundary():
    return 0.0

<div style="text-align: center;font-weight: bold;">表　暖房期の方位係数<br>Table: Orientation factor for heating season</div>

<table>
<thead>
<tr>
<th rowspan="2">方位<br>Orientation</th>
<th colspan="8">地域の区分<br>Segmentation of area</th>
</tr>
<tr>
<th>1</th>
<th>2</th>
<th>3</th>
<th>4</th>
<th>5</th>
<th>6</th>
<th>7</th>
<th>8</th>
</tr>
</thead>

<tbody>
<tr>
<th>上面<br>Top surface</th><td colspan="7" style="text-align: center;">1.0</td><td rowspan="10">-</td>
</tr>
<tr><th>北<br>North</th><td>0.260</td><td>0.263</td><td>0.284</td><td>0.256</td><td>0.238</td><td>0.261</td><td>0.227</td></tr>
<tr><th>北東<br>Northeast</th><td>0.333</td><td>0.341</td><td>0.348</td><td>0.330</td><td>0.310</td><td>0.325</td><td>0.281</td></tr>
<tr><th>東<br>East</th><td>0.564</td><td>0.554</td><td>0.540</td><td>0.531</td><td>0.568</td><td>0.579</td><td>0.543</td></tr>
<tr><th>南東<br>Southeast</th><td>0.823</td><td>0.766</td><td>0.751</td><td>0.724</td><td>0.846</td><td>0.833</td><td>0.843</td></tr>
<tr><th>南<br>South</th><td>0.935</td><td>0.856</td><td>0.851</td><td>0.815</td><td>0.983</td><td>0.936</td><td>1.023</td></tr>
<tr><th>南西<br>Southwest</th><td>0.790</td><td>0.753</td><td>0.750</td><td>0.723</td><td>0.815</td><td>0.763</td><td>0.848</td></tr>
<tr><th>西<br>West</th><td>0.535</td><td>0.544</td><td>0.542</td><td>0.527</td><td>0.538</td><td>0.523</td><td>0.548</td></tr>
<tr><th>北西<br>Northwest</th><td>0.325</td><td>0.341</td><td>0.351</td><td>0.326</td><td>0.297</td><td>0.317</td><td>0.284</td></tr>
<tr>
<th>下面<br>Bottom side</th><td colspan="7" style="text-align: center;">0.0</td>
</tr>
</tbody>
</table>


In [ ]:
# 外皮の部位の暖房期の方位係数
def get_nu_H(region, direction):
    """
    :param region: 地域の区分(1,2,3,4,5,6,7,8)
    :param direction: 外皮の部位の方位
    :return: 外皮の部位の暖房期の方位係数 (-)
    """
    # 表1 暖房期の方位係数
    table_1 = {
        '上面': (1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, None),
        '北': (0.260, 0.263, 0.284, 0.256, 0.238, 0.261, 0.227, None),
        '北東': (0.333, 0.341, 0.348, 0.330, 0.310, 0.325, 0.281, None),
        '東': (0.564, 0.554, 0.540, 0.531, 0.568, 0.579, 0.543, None),
        '南東': (0.823, 0.766, 0.751, 0.724, 0.846, 0.833, 0.843, None),
        '南': (0.935, 0.856, 0.851, 0.815, 0.983, 0.936, 1.023, None),
        '南西': (0.790, 0.753, 0.750, 0.723, 0.815, 0.763, 0.848, None),
        '西': (0.535, 0.544, 0.542, 0.527, 0.538, 0.523, 0.548, None),
        '北西': (0.325, 0.341, 0.351, 0.326, 0.297, 0.317, 0.284, None),
        '下面': (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, None),
    }
    return table_1[direction][region - 1]

<div style="text-align: center;font-weight: bold;">表　冷房期の方位係数<br>Table: Orientation coefficient in the cooling period</div>

<table>
<thead>
<tr>
<th rowspan="2">方位<br>Orientation</th>
<th colspan="8">地域の区分<br>Segmentation of area</th>
</tr>
<tr>
<th>1</th>
<th>2</th>
<th>3</th>
<th>4</th>
<th>5</th>
<th>6</th>
<th>7</th>
<th>8</th>
</tr>
</thead>

<tbody>
<tr>
<th>上面<br>Top surface</th><td colspan="8" style="text-align: center;">1.0</td>
</tr>
<tr><th>北<br>North</th><td>0.329</td><td>0.341</td><td>0.335</td><td>0.322</td><td>0.373</td><td>0.341</td><td>0.307</td><td>0.325</td></tr>
<tr><th>北東<br>Northeast</th><td>0.430</td><td>0.412</td><td>0.390</td><td>0.426</td><td>0.437</td><td>0.431</td><td>0.415</td><td>0.414</td></tr>
<tr><th>東<br>East</th><td>0.545</td><td>0.503</td><td>0.468</td><td>0.518</td><td>0.500</td><td>0.512</td><td>0.509</td><td>0.515</td></tr>
<tr><th>南東<br>Southeast</th><td>0.560</td><td>0.527</td><td>0.487</td><td>0.508</td><td>0.500</td><td>0.498</td><td>0.490</td><td>0.528</td></tr>
<tr><th>南<br>South</th><td>0.502</td><td>0.507</td><td>0.476</td><td>0.437</td><td>0.472</td><td>0.434</td><td>0.412</td><td>0.480</td></tr>
<tr><th>南西<br>Southwest</th><td>0.526</td><td>0.548</td><td>0.550</td><td>0.481</td><td>0.520</td><td>0.491</td><td>0.479</td><td>0.517</td></tr>
<tr><th>西<br>West</th><td>0.508</td><td>0.529</td><td>0.553</td><td>0.481</td><td>0.518</td><td>0.504</td><td>0.495</td><td>0.505</td></tr>
<tr><th>北西<br>Northwest</th><td>0.411</td><td>0.428</td><td>0.447</td><td>0.401</td><td>0.442</td><td>0.427</td><td>0.406</td><td>0.411</td></tr>
<tr>
<th>下面<br>Bottom side</th><td colspan="8" style="text-align: center;">0.0</td>
</tr>
</tbody>
</table>


In [ ]:
# 外皮の部位の冷房期の方位係数
def get_nu_C(region, direction):
    """
    :param region: 地域の区分(1,2,3,4,5,6,7,8)
    :param direction: 外皮の部位の方位
    :return: 外皮の部位の暖房期の方位係数 (-)
    """
    # 表2 冷房期の方位係数
    table_2 = {
        '上面': (1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0),
        '北': (0.329, 0.341, 0.335, 0.322, 0.373, 0.341, 0.307, 0.325),
        '北東': (0.430, 0.412, 0.390, 0.426, 0.437, 0.431, 0.415, 0.414),
        '東': (0.545, 0.503, 0.468, 0.518, 0.500, 0.512, 0.509, 0.515),
        '南東': (0.560, 0.527, 0.487, 0.508, 0.500, 0.498, 0.490, 0.528),
        '南': (0.502, 0.507, 0.476, 0.437, 0.472, 0.434, 0.412, 0.480),
        '南西': (0.526, 0.548, 0.550, 0.481, 0.520, 0.491, 0.479, 0.517),
        '西': (0.508, 0.529, 0.553, 0.481, 0.518, 0.503, 0.495, 0.505),
        '北西': (0.411, 0.428, 0.447, 0.401, 0.442, 0.427, 0.406, 0.411),
        '下面': (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
    }
    return table_2[direction][region - 1]